# Лабораторная работа №2

Требования:
* Python >= 3.X
* NLTK >= 3.2.5

Лабораторную работу необходимо выполнять в данном шаблоне. Результат работы выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)). Откликами являются значения двух классов positive и negative.

In [38]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip  /content/drive/MyDrive/Datasets/IMDB_Dataset_of_50K_Movie_Reviews.zip -d data

Archive:  /content/drive/MyDrive/Datasets/IMDB_Dataset_of_50K_Movie_Reviews.zip
  inflating: data/IMDB Dataset.csv   


In [55]:
frame = pd.read_csv('/content/data/IMDB Dataset.csv', encoding = 'utf8')[:1500]
frame.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Шаг 1. Подготовка данных

В качестве исследуемых способов представления необходимо рассмотреть:
#### 1. Bag of words: word counts ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)). Компоненты вектора: частоты или относительные частоты.

In [56]:
X = frame['review']
y = (frame['sentiment'] == 'negative').astype(int)

In [57]:
cv = CountVectorizer()
X1 = cv.fit_transform(X).toarray()
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.5, random_state=0)
print('cv_train:', X1_train.shape)
print('cv_test:', X1_test.shape)

cv_train: (750, 21653)
cv_test: (750, 21653)


In [58]:
type(X1)

numpy.ndarray

#### 2. Bag of words: weird numbers ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). Компоненты вектора: оценки "значимости" слова (например tf-idf).

In [59]:
#Tfidf vectorizer
tv=TfidfVectorizer()
#transformed reviews
X2=tv.fit_transform(X).toarray()
X2_train, X2_test = train_test_split(X1, test_size=0.5, random_state=0)
print('Tfidf_train:',X2_train.shape)
print('Tfidf_test:',X2_test.shape)

Tfidf_train: (750, 21653)
Tfidf_test: (750, 21653)


In [60]:
type(X2_test)

numpy.ndarray

#### 3. Bag of ngrams. Позволяет учитывать положение слов. Компоненты вектора: частоты N-грам. Примитивный подход, но в некоторых задачах он может улучшать качество решения.

In [61]:
#Count vectorizer for bag of words
cv=CountVectorizer(ngram_range=(2,2))
#transformed reviews

X3 = cv.fit_transform(X).toarray()
X3_train, X3_test = train_test_split(X2, test_size=0.5, random_state=0)

print('BOW_cv_train:',X3_train.shape)
print('BOW_cv_test:',X3_test.shape)

BOW_cv_train: (750, 21653)
BOW_cv_test: (750, 21653)


In [62]:
type(X3)

numpy.ndarray

### Шаг 2. Исследование моделей

В зависимости от способа представления оценить качество классификации как долю правильных ответов на выборке (accuracy). Не забывайте использовать перекрестную проверку ([cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)).

Для каждого из нижеперечисленных моделей необходимо определить оптимальные гиперпараметры ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

Качество классификации оцениваем для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

In [63]:
svc = SVC()
param_grid = {'kernel': ('linear', 'rbf'), 'C': [1, 10]}

clf_cv = GridSearchCV(svc, param_grid)
clf_cv.fit(X1_train, y_train)
clf_tv = GridSearchCV(svc, param_grid)
clf_tv.fit(X2_train, y_train)
clf_ngram = GridSearchCV(svc, param_grid)
clf_ngram.fit(X2_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

In [64]:
param_grid = {
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}
rfc = RandomForestClassifier()

clf_cv_f = GridSearchCV(rfc, param_grid)
clf_cv_f.fit(X1_train, y_train)

clf_tv_f = GridSearchCV(rfc, param_grid)
clf_tv_f.fit(X2_train, y_train)

clf_ngram_f = GridSearchCV(rfc, param_grid)
clf_ngram_f.fit(X3_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 700]})

### Шаг 3. Сравнение результатов

Сравнить точность обученных моделей. Найти наиболее точную.

In [67]:
clfs = {"SVC_word counts": clf_cv,
        "SVC_wierd_numbers":clf_tv, 
        "SVC_bag_of_ngrams":clf_ngram,
        "RFC_word counts": clf_cv_f, 
        "RFC_wierd_numbers":clf_tv_f, 
        "RFC_bag_of_ngrams":clf_ngram_f
        }

test_vals = {"SVC_word counts": X1_test, 
             "SVC_wierd_numbers":X2_test, 
             "SVC_bag_of_ngrams":X3_test,
             "RFC_word counts": X1_test, 
             "RFC_wierd_numbers":X2_test, 
             "RFC_bag_of_ngrams":X3_test
             }

for a, b in clfs.items():
    print(a)
    estimator = b.best_estimator_
    y_pred = estimator.predict(test_vals[a])
    print(accuracy_score(y_test,y_pred))

SVC_word counts
0.776
SVC_wierd_numbers
0.776
SVC_bag_of_ngrams
0.49466666666666664
RFC_word counts
0.788
RFC_wierd_numbers
0.7853333333333333
RFC_bag_of_ngrams
0.7693333333333333
